In [84]:
from collections import namedtuple
import hashlib

Node = namedtuple('nodes', ['id'])
Edge = namedtuple('edge', ['id', 'src', 'dst', 'length'])

In [85]:
with open('citymapper-coding-test-graph.dat', 'r') as f:
    data = f.readlines()

In [86]:
nodes = []
edges = []

# Number of vertices
N = int(data[0].strip("\n"))
for i in range(1, N+1):
    nodes.append(Node(int(data[i].strip("\n"))))

# Number of edges
E = int(data[N+1])
for i in range(N+2, E+1):
    src, dst, length = data[i].strip("\n").split(" ")
    e_id = hashlib.sha256("{}_{}".format(src,dst).encode()).hexdigest()
    edges.append(Edge(e_id, int(src), int(dst), int(length)))

[nodes(id=1084140204),
 nodes(id=319946794),
 nodes(id=319946795),
 nodes(id=319946796),
 nodes(id=319946799),
 nodes(id=319946800),
 nodes(id=36306993),
 nodes(id=36306994),
 nodes(id=36306995),
 nodes(id=36306996),
 nodes(id=36306997),
 nodes(id=36306998),
 nodes(id=36306999),
 nodes(id=36307000),
 nodes(id=36307001),
 nodes(id=36307002),
 nodes(id=36307003),
 nodes(id=36307004),
 nodes(id=36307005),
 nodes(id=876500321),
 nodes(id=447086666),
 nodes(id=36328802),
 nodes(id=447086672),
 nodes(id=1001433784),
 nodes(id=36328803),
 nodes(id=36328804),
 nodes(id=36307039),
 nodes(id=36307040),
 nodes(id=36307042),
 nodes(id=36307043),
 nodes(id=36307044),
 nodes(id=876500326),
 nodes(id=36307046),
 nodes(id=36307049),
 nodes(id=36307051),
 nodes(id=36307053),
 nodes(id=36307056),
 nodes(id=319946865),
 nodes(id=319946866),
 nodes(id=319946867),
 nodes(id=319946869),
 nodes(id=319946870),
 nodes(id=36307063),
 nodes(id=36307067),
 nodes(id=316276861),
 nodes(id=36307070),
 nodes(id=36307